In [ ]:
def compute_boundary_distances(sec1, sec2):
    
    dm1 = DataManager(generate_hierarchy=False, stack='RS141', resol='x5', section=sec1)
    dm2 = DataManager(generate_hierarchy=False, stack='RS141', resol='x5', section=sec2)
    
    boundaries1 = dm1.load_pipeline_result('boundaryModels', 'pkl')
    boundaries2 = dm2.load_pipeline_result('boundaryModels', 'pkl')
    
    sc1 = dm1.load_pipeline_result('shapeContext', 'pkl')
    sc2 = dm2.load_pipeline_result('shapeContext', 'pkl')

    n_boundaries1 = len(boundaries1)
    n_boundaries2 = len(boundaries2)

    Ds = Parallel(n_jobs=16)(delayed(boundary_distance)(boundaries1[i], boundaries2[j], sc1=sc1[i], sc2=sc2[j]) 
                             for i, j in product(range(n_boundaries1), range(n_boundaries2)))
    D_boundaries, D_int, D_ext, D_shape, D_loc = zip(*Ds)

    D_boundaries = np.reshape(D_boundaries, (n_boundaries1, n_boundaries2))
    D1 = np.reshape(D_int, (n_boundaries1, n_boundaries2))
    D2 = np.reshape(D_ext, (n_boundaries1, n_boundaries2))
    D3 = np.reshape(D_shape, (n_boundaries1, n_boundaries2))
    D4 = np.reshape(D_loc, (n_boundaries1, n_boundaries2))
    
    dm1.save_pipeline_result(D_boundaries, 'DBoundaries%dWith%d'%(sec1, sec2), 'npy')
    dm1.save_pipeline_result(D1, 'D1s%dWith%d'%(sec1, sec2), 'npy')
    dm1.save_pipeline_result(D2, 'D2s%dWith%d'%(sec1, sec2), 'npy')
    dm1.save_pipeline_result(D3, 'D3s%dWith%d'%(sec1, sec2), 'npy')
    dm1.save_pipeline_result(D4, 'D4s%dWith%d'%(sec1, sec2), 'npy')

    dm2.save_pipeline_result(D_boundaries.T, 'DBoundaries%dWith%d'%(sec2, sec1), 'npy')
    dm2.save_pipeline_result(D1.T, 'D1s%dWith%d'%(sec2, sec1), 'npy')
    dm2.save_pipeline_result(D2.T, 'D2s%dWith%d'%(sec2, sec1), 'npy')
    dm2.save_pipeline_result(D3.T, 'D3s%dWith%d'%(sec2, sec1), 'npy')
    dm2.save_pipeline_result(D4.T, 'D4s%dWith%d'%(sec2, sec1), 'npy')


In [ ]:
import argparse

if __name__ == '__main__':
    
    parser = argparse.ArgumentParser(
        formatter_class=argparse.RawDescriptionHelpFormatter,
        description='Match boundaries between section n with n+1, n+2 and n+3')

    parser.add_argument("stack_name", type=str, help="stack name")
    parser.add_argument("slice_ind", type=int, help="slice index")
    parser.add_argument("-g", "--gabor_params_id", type=str, help="gabor filter parameters id (default: %(default)s)", default='blueNisslWide')
    parser.add_argument("-s", "--segm_params_id", type=str, help="segmentation parameters id (default: %(default)s)", default='blueNisslRegular')
    parser.add_argument("-v", "--vq_params_id", type=str, help="vector quantization parameters id (default: %(default)s)", default='blueNissl')
    args = parser.parse_args()

    sec_ind = int(args.slice_ind)
    
    
    
    compute_boundary_distances(sec_ind)